In [25]:
import numpy as np, pandas as pd

In [69]:
class GDA(object):
    
    def __init__(self, features, target):
        self.features=features
        self.target=target
        
    def train(self):    
        self.mu0=self.features.iloc[[i for i in range(self.target.shape[0]) if \
                                     self.target.iloc[i]==0.0]].mean()
        self.mu1=self.features.iloc[[i for i in range(self.target.shape[0]) if \
                                     self.target.iloc[i]==1.0]].mean()
        
        self.phi=(1.0/self.target.shape[0])*self.target[self.target==1.0].count()
        
        sigma=np.matrix(np.zeros([self.features.shape[1], self.features.shape[1]]))
        for i in range(self.target.shape[0]):
            if self.target.iloc[i]==0:
                sigma += np.dot(np.matrix(self.features.iloc[i, :]-self.mu0).T, \
                                np.matrix(self.features.iloc[i, :]-self.mu0))
            else:
                sigma += np.dot(np.matrix(self.features.iloc[i, :]-self.mu1).T,\
                                np.matrix(self.features.iloc[i, :]-self.mu1))
        self.sigma=(1.0/self.target.shape[0])*sigma
        
    def P_y(self, y, phi):
        return phi**y * (1-phi)**(1-y)
    
    def P_x_y(self, sigma, x, mu):
        comp1 = 1.0/(np.sqrt((2*np.pi)**self.features.shape[1]) * np.sqrt(np.linalg.det(sigma)))
        comp2 = float(np.exp(np.dot(-0.5*np.dot(x-mu, np.linalg.inv(sigma)), x-mu)))
        return comp1*comp2
        
    def predict(self, X):
        predictions=[]
        for i in range(X.shape[0]):
            Prob0=self.P_x_y(self.sigma, X.iloc[i, :], self.mu0)*self.P_y(0, self.phi)
            Prob1=self.P_x_y(self.sigma, X.iloc[i, :], self.mu1)*self.P_y(1, self.phi)
            if Prob0>Prob1:
                predictions.append(0.0)
            else:
                predictions.append(1.0)
        return np.array(predictions)
            

In [178]:
from sklearn import datasets

Iris=datasets.load_iris()
data=Iris['data']
Data=np.hstack([data, Iris['target'].reshape(-1,1)])
Random=list(range(data.shape[0]))
np.random.shuffle(Random)
Data=Data[Random]
Col=Iris['feature_names']
Col.insert(len(Col), 'target')
Data=pd.DataFrame(Data, columns=Col)
Data.head()
Data=Data[(Data['target']==0) | (Data['target']==1)]

Train_data=Data.iloc[0:10]
Test_data=Data.iloc[10:]
print("Train data is 10% of the data ",Train_data.shape)
print("Test data is 90% of the data ",Test_data.shape)
print("100% accuracy on the test set")

Train data is 10% of the data  (10, 5)
Test data is 90% of the data  (90, 5)
100% accuracy on the test set


In [179]:
GaussianIris=GDA(Train_data.iloc[:,:-1], Train_data.iloc[:,-1])

GaussianIris.train()

pred=GaussianIris.predict(Test_data.iloc[:,:-1])
pred=pred.reshape(-1,1)
print("Accuracy: ", (pred==np.array(Test_data.iloc[:,-1]).reshape(-1,1)).sum()/Test_data.iloc[:,-1].shape[0]*100, "%")

Accuracy:  100.0 %
